Let's do a simpler version of torch choice based on an example in Warren Kuhfeld's SAS book

https://support.sas.com/techsup/technote/mr2010.pdf

Candy example 

In [1]:
import pandas as pd
import numpy as np
import torch
from torch_choice.utils.easy_data_wrapper import EasyDatasetWrapper
from torch_choice.model import ConditionalLogitModel
from torch_choice import run
from skimpy import skim

if torch.cuda.is_available():
    device = 'cuda'  # use GPU if available
else:
    device = 'cpu'  # use CPU otherwise

In [2]:
# Read the candy data
candy = pd.read_csv("./data/candy.csv")

In [3]:
# Inspect the data
display(candy)

,subj,set,c,dark,soft,nuts
0,1,1,2,0,0,0
1,1,1,1,1,0,0
2,1,1,2,0,0,1
3,1,1,2,1,0,1
4,1,1,2,0,1,0
...,...,...,...,...,...,...
75,10,1,1,1,0,1
76,10,1,2,0,1,0
77,10,1,2,1,1,0
78,10,1,2,0,1,1


The selection column isn't in the right format (as it is prepped to use PROC PHREG in SAS), let's fix that

In [4]:
candy['c'] = 2 - candy['c']
candy['option'] = np.tile(np.arange(1, 9), 10)
candy['unique_cs'] = candy['subj'] * 10 + candy['set']
display(candy)

,subj,set,c,dark,soft,nuts,option,unique_cs
0,1,1,0,0,0,0,1,11
1,1,1,1,1,0,0,2,11
2,1,1,0,0,0,1,3,11
3,1,1,0,1,0,1,4,11
4,1,1,0,0,1,0,5,11
...,...,...,...,...,...,...,...,...
75,10,1,1,1,0,1,4,101
76,10,1,0,0,1,0,5,101
77,10,1,0,1,1,0,6,101
78,10,1,0,0,1,1,7,101


Now we should be able to convert to a choice set object

In [8]:
candy_choice = EasyDatasetWrapper(
    main_data = candy,
    purchase_record_column = 'unique_cs',
    choice_column = 'c',
    item_name_column =  'option',
    user_index_column = 'subj',
    item_observable_columns = ['dark', 'soft', 'nuts'],
    num_items=8,
    device = device)

candy_choice.summary()
candy_choice_data = candy_choice.choice_dataset

Creating choice dataset from stata format data-frames...
No `session_index` is provided, assume each choice instance is in its own session.
Finished Creating Choice Dataset.
* purchase record index range: [11 21 31] ... [ 81  91 101]
* Space of 8 items:
            0  1  2  3  4  5  6  7
item name  1  2  3  4  5  6  7  8
* Number of purchase records/cases: 10.
* Preview of main data frame:
    subj  set  c  dark  soft  nuts  option  unique_cs
0      1    1  0     0     0     0       1         11
1      1    1  1     1     0     0       2         11
2      1    1  0     0     0     1       3         11
3      1    1  0     1     0     1       4         11
4      1    1  0     0     1     0       5         11
..   ...  ... ..   ...   ...   ...     ...        ...
75    10    1  1     1     0     1       4        101
76    10    1  0     0     1     0       5        101
77    10    1  0     1     1     0       6        101
78    10    1  0     0     1     1       7        101
79    10    1

C:\Users\steph\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch_choice\data\choice_dataset.py:215: UserWarning: The number of users is inferred from the number of unique users in the user_index tensor. This might lead to unexpected behaviors if some users never appeared in the user_index tensor. For a safer behavior, please provide the number of users explicitly by using the num_users keyword while initializing the ChoiceDataset class.
  warnings.warn(f"The number of users is inferred from the number of unique users in the user_index tensor. This might lead to unexpected behaviors if some users never appeared in the user_index tensor. For a safer behavior, please provide the number of users explicitly by using the num_users keyword while initializing the ChoiceDataset class.")
C:\Users\steph\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-p

In [11]:
model = ConditionalLogitModel(
    formula='(item_dark|constant) + (item_soft|constant) + (item_nuts|constant)',
    dataset=candy_choice_data,
    num_items=8)

In [12]:
run(model, candy_choice_data, num_epochs=500, learning_rate=0.01, model_optimizer="LBFGS", batch_size=-1)

==================== model received ====================
ConditionalLogitModel(
  (coef_dict): ModuleDict(
    (item_dark[constant]): Coefficient(variation=constant, num_items=8, num_users=None, num_params=1, 1 trainable parameters in total, initialization=normal, device=cpu).
    (item_soft[constant]): Coefficient(variation=constant, num_items=8, num_users=None, num_params=1, 1 trainable parameters in total, initialization=normal, device=cpu).
    (item_nuts[constant]): Coefficient(variation=constant, num_items=8, num_users=None, num_params=1, 1 trainable parameters in total, initialization=normal, device=cpu).
  )
)
Conditional logistic discrete choice model, expects input features:

X[item_dark[constant]] with 1 parameters, with constant level variation.
X[item_soft[constant]] with 1 parameters, with constant level variation.
X[item_nuts[constant]] with 1 parameters, with constant level variation.
device=cpu
==================== data set received ====================
[Train dataset]

C:\Users\steph\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch_choice\data\choice_dataset.py:215: UserWarning: The number of users is inferred from the number of unique users in the user_index tensor. This might lead to unexpected behaviors if some users never appeared in the user_index tensor. For a safer behavior, please provide the number of users explicitly by using the num_users keyword while initializing the ChoiceDataset class.
  warnings.warn(f"The number of users is inferred from the number of unique users in the user_index tensor. This might lead to unexpected behaviors if some users never appeared in the user_index tensor. For a safer behavior, please provide the number of users explicitly by using the num_users keyword while initializing the ChoiceDataset class.")
C:\Users\steph\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-p

Epoch 499: 100%|██████████| 1/1 [00:00<00:00, 24.76it/s, v_num=13]  

`Trainer.fit` stopped: `max_epochs=500` reached.


Epoch 499: 100%|██████████| 1/1 [00:00<00:00, 23.05it/s, v_num=13]
Time taken for training: 13.668880462646484
Skip testing, no test dataset is provided.
==================== model results ====================
Log-likelihood: [Training] -14.363497734069824, [Validation] N/A, [Test] N/A

| Coefficient           |   Estimation |   Std. Err. |   z-value |   Pr(>|z|) | Significance   |
|:----------------------|-------------:|------------:|----------:|-----------:|:---------------|
| item_dark[constant]_0 |     1.38627  |    0.790565 |   1.75352 |  0.079512  |                |
| item_soft[constant]_0 |    -2.19721  |    1.05409  |  -2.08447 |  0.0371176 | *              |
| item_nuts[constant]_0 |     0.847289 |    0.690064 |   1.22784 |  0.219507  |                |
Significance codes: 0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


ConditionalLogitModel(
  (coef_dict): ModuleDict(
    (item_dark[constant]): Coefficient(variation=constant, num_items=8, num_users=None, num_params=1, 1 trainable parameters in total, initialization=normal, device=cpu).
    (item_soft[constant]): Coefficient(variation=constant, num_items=8, num_users=None, num_params=1, 1 trainable parameters in total, initialization=normal, device=cpu).
    (item_nuts[constant]): Coefficient(variation=constant, num_items=8, num_users=None, num_params=1, 1 trainable parameters in total, initialization=normal, device=cpu).
  )
)
Conditional logistic discrete choice model, expects input features:

X[item_dark[constant]] with 1 parameters, with constant level variation.
X[item_soft[constant]] with 1 parameters, with constant level variation.
X[item_nuts[constant]] with 1 parameters, with constant level variation.
device=cpu

This matches the output in the SAS book.